In [1]:
import pandas as pd
import numpy as np
import re
read_file = pd.read_excel (r'collection_matched.xlsx')
read_file.to_csv (r'collection_matched.csv', index = None, header=True)

In [2]:
sample = pd.read_csv('collection_matched.csv')


In [3]:
sample['TextContent'][149]

'CITIZENS HOLDING COMPANY \n521 Main St., Philadelphia, MS 39350 \nTel; 601-656-4692 \nhttp:/ /www.citizensholdingcompany.com \n*Chrni, Pres & Chief Exec Officer-Joe Steve Webb \nTreas-Robert T. Smith \nSecy-Carolyn K. McKee \nAccts- Horne CPA Group, Jackson, MS \nPrimary Bank-Trustmark National Bank \nPrimary Law Firm-Alford Thomas & Kilgore \nNet Sales: S31.90Mil Employees: 131 \nStock Exchange(s): ASE Ticker: CIZ \nOther Directors Are: \nM. G. Bond \nW.W. Dungan \nDon L. Kilgore \nHerbert A. King \nWilliam M. Mars \nDavid P. Webb \nKarl Brantley \nDon L. Fulton \nDavid A. King \nGeorge R. · Mars \nGreg McKee \nBUSINESS: Bank holding co. \nNAIC: 551111 '

In [4]:
#check if there is a number in text
def num_there(text):
    return any(i.isdigit() for i in text)

In [5]:
#function for extracting the name of law firm
def extract_PLF(text):
    trial = text.split('Primary Law Firm')[1]
    splited = trial.split("\n")
    second_sentence = splited[1]
    third_sentence = splited[2]
    if num_there(second_sentence) is True:
        law_firm= splited[0]
        #print (law_firm)
    elif num_there(second_sentence) is False and num_there(third_sentence) is True:
        law_firm= splited[0]+splited[1]
    elif num_there(second_sentence) is False and num_there(third_sentence) is False:
        law_firm = splited[0]+splited[1]+splited[2]
    return law_firm

In [6]:
stopwords2 = [

"Total Assets","Total Net Assets","Net Assets","Gross Sales","Net Sales","Combined Sales","Gross Oper. Revenue",
"Gross Oper. Rev.","Gross Oper Revenue","Gross Oper Rev.","Gross Oper Rev","Gross Operating Rev.",
'Gross Oper Ell!"Jls:',"Total Oper. Rev.","Total Revenues","Total Revenue","Consol Revenue","Gross Revenue",
"Gross Revs.","Net Revenue","Oper Revenues","Oper. Revenue","Oper Revenue","Prod Revenue",
"Gross Oper. Earns","Gross Oper Earns","Gross Oper Earn","Net Earnings","Net Oper Earns",
"Total Income","Gross Income","Operating Income","Oper. Income","Total Net Income","Net.Income",
"Net Income","Netlncome","Total lat. Income","Total Investment Income","Net Investment Income",
"Net Investment Inc.","Net Invest. Income","Netlnvestmentlnc.","Net Int. Income","Net Int. btcome:",
"Net Investment","Investment Income","Invest. Income Rev.","Invest Income","Net Interest Income",
"Total Interest Incom","Total Int. Income","Total Int. Incom","Total Intrest Incol!le:",
"Total Intrest Incol!le:","Interest Income","Interest Revenue","Ann. Premium Incom","Royalties & Contract",
"Mutuel Play","Total Deposits","Gross Deposits","Gross Deposit","Deposits:","Assets","Asset",
"Sales","Sale","sales","SaJes:","Saies:","Sa_les:","Sal;~~:","Sal;~~","S3les:","Sates:","Revenues",
"Revenue","ReYenue:", "Revenne","Revenae:","Reven.ue:","R~venue:","Reven~e:","Reven11e:","Re_venue:",
"R~venue:","Reven_ue:","Jlevenue:","Eams:","Income","Premiums","Premium","Employees","~ployees:",
"17840amings:","Stock Exchange","Stock Excliange","Stock Exohange","Stock Ezchange","Stock Ex~ange",
"Ticker","Also DIRECTORS","Other Directors","Directors:","Directors Are:","Canadian:","PRODUCTS","BUSINESS"
]

In [7]:
#final extraction algorithm
text = "Kwiatt & Silvennan, Ltd. BUSINESS: Holding company"
for stopword in stopwords2:
            # If included in a stopword, stop the loop and return text before the stopword
    if (stopword in text):
                # Get the text before the word
        word = text.split(stopword)[0]
                # If the split[0] is empty, no need to include in the return text
        break
print(word)
        

Kwiatt & Silvennan, Ltd. 


In [8]:
def clean_text2(text, stopwords2):
    #text = text.split('Primary Law Firm')[1]
    for stopword in stopwords2:
        if (stopword in text):
            text = text[:re.search(stopword, text).start()]
    return text

In [9]:
names= []
n= len(sample.axes[0])
for row in range (n):
    text_content = sample['TextContent'][row]
    law_firm = extract_PLF(text_content)
    this_company = clean_text2(law_firm, stopwords2)
    names.append(this_company)
#print(names)

In [10]:
len(names)

9088

In [11]:
cleaned_names = []
for i in names:
    if '--' in i:
        cleaned_names.append(i[2:])
    elif '-' in i:
        cleaned_names.append(i[1:])   
    elif '~' in i:
        cleaned_names.append(i[1:])
    else:
        cleaned_names.append(i[:])
#cleaned_names


In [12]:
len(cleaned_names)

9088

In [13]:
data= [cleaned_names[i] for i in range(len(cleaned_names))]
df = pd.DataFrame(data) 

In [14]:
df.head

<bound method NDFrame.head of                                                       0
0                    ".Mackall, ._Crounse & Moote, PLC 
1                 Benesch, Friedlander, Cop~ & Aronoff 
2                          Morgan, Lewis-& Bockius LLP 
3                                     Quarles -& Brady 
4                                     Baker- & Daniels 
...                                                 ...
9083    Zuckerman, Spaeder, Goldstein, Taylor & Kolker 
9084  Zuckerman, Spaeder, Goldstein, Taylor & Kolker...
9085                     Zuckert, Scoutt & Rasenberger 
9086                     Zukerman Gore & Brandeis, LLP 
9087             ---Z:ullen & Dykman Bleakey Platt LLP 

[9088 rows x 1 columns]>

In [15]:
df.to_excel (r'collection_matched_number+stopwords.xlsx', index = None, header=True)